In [67]:
import googlemaps
import time
import dotenv
import os
from pprintpp import pprint
import pandas as pd

import folium
from folium.plugins import HeatMap, MarkerCluster


%load_ext dotenv
chave_api = os.getenv("API_KEY")
client = googlemaps.Client(chave_api)

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [47]:
meu_endereco = 'Estrada Sta Maria, rio de janeiro, rj, brasil'

minha_coord = client.geocode(meu_endereco)[0]['geometry']['location']
minha_coord #dict

{'lat': -22.8834606, 'lng': -43.5800678}

In [50]:
next_page_token = ''


lista_coordenadas = []
response = client.places_nearby(keyword= 'Mc Donalds', radius=5000, location= minha_coord)
while True:
    for store in response['results']:
        latitude = store['geometry']['location']['lat']
        longitude = store['geometry']['location']['lng']

        coordenadas = (latitude, longitude)
        lista_coordenadas.append(coordenadas)

    if 'next_page_token' in response:
        time.sleep(2)
        next_page_token = response['next_page_token']
        response = client.places(query='Mc Donalds', page_token= next_page_token, radius=5000, location= minha_coord)

    else:
        break

In [61]:
print(list(minha_coord.values()))
mapa = folium.Map(location= list(minha_coord.values()), zoom_start= 15)
HeatMap(lista_coordenadas, radius= 30).add_to(mapa)
mapa

[-22.8834606, -43.5800678]


In [116]:
nome_estado = 'rio de janeiro, brasil'

estado_coord = list(client.geocode(nome_estado)[0]['geometry']['location'].values())
estado_coord

[-22.9068467, -43.1728965]

In [184]:
lista_dfs = []

response = client.places_nearby(location=estado_coord, keyword="Mc Donalds", radius=50000)

while True:
    time.sleep(0.1)

    for j, store in enumerate(response["results"]):
        print(f'{j}/{len(response["results"])} - response')

        df = pd.DataFrame(
            {
                "nome": store["name"],
                "endereco": store["vicinity"],
                "latitude": store["geometry"]["location"]["lat"],
                "longitude": store["geometry"]["location"]["lng"],
            },
            index=[0],
        )
        lista_dfs.append(df)

    if "next_page_token" in response:
        next_page_token = response["next_page_token"]
        time.sleep(2.3)
        response = client.places_nearby(
            location=coordenada,
            keyword="Riachuelo",
            radius=50000,
            page_token=next_page_token,
        )

    else:
        break

lojas_mc = pd.concat(lista_dfs, ignore_index=True)

0/20 - response
1/20 - response
2/20 - response
3/20 - response
4/20 - response
5/20 - response
6/20 - response
7/20 - response
8/20 - response
9/20 - response
10/20 - response
11/20 - response
12/20 - response
13/20 - response
14/20 - response
15/20 - response
16/20 - response
17/20 - response
18/20 - response
19/20 - response
0/20 - response
1/20 - response
2/20 - response
3/20 - response
4/20 - response
5/20 - response
6/20 - response
7/20 - response
8/20 - response
9/20 - response
10/20 - response
11/20 - response
12/20 - response
13/20 - response
14/20 - response
15/20 - response
16/20 - response
17/20 - response
18/20 - response
19/20 - response
0/20 - response
1/20 - response
2/20 - response
3/20 - response
4/20 - response
5/20 - response
6/20 - response
7/20 - response
8/20 - response
9/20 - response
10/20 - response
11/20 - response
12/20 - response
13/20 - response
14/20 - response
15/20 - response
16/20 - response
17/20 - response
18/20 - response
19/20 - response


In [189]:
lojas_mc_filtrado = lojas_mc[
    (lojas_mc["nome"].str.lower().str.contains("mcdonald"))
    | lojas_mc["nome"].str.lower().str.contains("mc donald")
]
mc_estado_coord = lojas_mc_filtrado[['latitude', 'longitude']].values

In [198]:
map_calor = folium.Map(location=estado_coord, zoom_start=8, tiles="Cartodb dark_matter",)

HeatMap(mc_estado_coord, radius= 30).add_to(map_calor)

map_calor